In [1]:
from config import settings
from src.partition import Partition
from src.generators import PartitionClass
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from keras.models import Sequential, Model, load_model
from keras.layers import (
    Dense, Conv2D, Flatten, 
    ConvLSTM2D, BatchNormalization,
    Conv3D, Input, MaxPooling2D, UpSampling2D,
)

from keras.callbacks.callbacks import EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


# Keras Stability Model

In [7]:
n = 28

In [8]:
training_partitions = [p for p in PartitionClass(n).grid_group(n) if not p.is_stable]
training_matrices = [p.fit_matrix(n) for p in training_partitions]
training_targets = [p._next_oblak_step[1].fit_matrix(n) for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_targets)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
y_train = y_train.reshape(len(y_train), n, n, 1)
y_test = y_test.reshape(len(y_test), n, n, 1)

In [9]:
input_img = Input(shape=(n, n, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 8)           0   

In [10]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.001, patience=5000, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [11]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10000, batch_size=10, workers = 4)

Train on 13339 samples, validate on 4447 samples
Epoch 1/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0173 - accuracy: 0.9940 - val_loss: 0.0100 - val_accuracy: 0.9957
Epoch 2/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0092 - accuracy: 0.9960 - val_loss: 0.0085 - val_accuracy: 0.9965
Epoch 3/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0078 - accuracy: 0.9967 - val_loss: 0.0073 - val_accuracy: 0.9969
Epoch 4/10000
13339/13339 [==============================] - 17s 1ms/step - loss: 0.0071 - accuracy: 0.9970 - val_loss: 0.0068 - val_accuracy: 0.9971
Epoch 5/10000
13339/13339 [==============================] - 20s 1ms/step - loss: 0.0067 - accuracy: 0.9972 - val_loss: 0.0066 - val_accuracy: 0.9972
Epoch 6/10000
13339/13339 [==============================] - 17s 1ms/step - loss: 0.0063 - accuracy: 0.9973 - val_loss: 0.0061 - val_accuracy: 0.9974
Epoch 7/10000
13339/13339 [========================

Epoch 55/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0019 - accuracy: 0.9993 - val_loss: 0.0020 - val_accuracy: 0.9992
Epoch 56/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0019 - accuracy: 0.9993 - val_loss: 0.0019 - val_accuracy: 0.9993
Epoch 57/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0019 - accuracy: 0.9993 - val_loss: 0.0025 - val_accuracy: 0.9991
Epoch 58/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0019 - accuracy: 0.9993 - val_loss: 0.0019 - val_accuracy: 0.9993
Epoch 59/10000
13339/13339 [==============================] - 18s 1ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 0.0020 - val_accuracy: 0.9992
Epoch 60/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 0.0022 - val_accuracy: 0.9992
Epoch 61/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0018 - ac

13339/13339 [==============================] - 20s 1ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0013 - val_accuracy: 0.9995
Epoch 110/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0013 - val_accuracy: 0.9996
Epoch 111/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0013 - val_accuracy: 0.9996
Epoch 112/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0014 - val_accuracy: 0.9995
Epoch 113/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0014 - val_accuracy: 0.9995
Epoch 114/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0013 - val_accuracy: 0.9995
Epoch 115/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 0.0011 - accuracy: 0

Epoch 162/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 8.1953e-04 - accuracy: 0.9997 - val_loss: 9.7747e-04 - val_accuracy: 0.9997
Epoch 163/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 7.9787e-04 - accuracy: 0.9997 - val_loss: 0.0013 - val_accuracy: 0.9996
Epoch 164/10000
13339/13339 [==============================] - 20s 1ms/step - loss: 7.7396e-04 - accuracy: 0.9997 - val_loss: 0.0012 - val_accuracy: 0.9996
Epoch 165/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 8.2268e-04 - accuracy: 0.9997 - val_loss: 0.0019 - val_accuracy: 0.9995
Epoch 166/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 8.0929e-04 - accuracy: 0.9997 - val_loss: 0.0011 - val_accuracy: 0.9996
Epoch 167/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 7.8007e-04 - accuracy: 0.9997 - val_loss: 0.0010 - val_accuracy: 0.9997
Epoch 168/10000
13339/13339 [==============================]

13339/13339 [==============================] - 19s 1ms/step - loss: 6.3080e-04 - accuracy: 0.9998 - val_loss: 9.8929e-04 - val_accuracy: 0.9997
Epoch 215/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 6.7232e-04 - accuracy: 0.9998 - val_loss: 9.0788e-04 - val_accuracy: 0.9997
Epoch 216/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 6.7282e-04 - accuracy: 0.9998 - val_loss: 0.0011 - val_accuracy: 0.9997
Epoch 217/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 6.6793e-04 - accuracy: 0.9998 - val_loss: 0.0011 - val_accuracy: 0.9997
Epoch 218/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 6.3943e-04 - accuracy: 0.9998 - val_loss: 9.3058e-04 - val_accuracy: 0.9997
Epoch 219/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 6.2916e-04 - accuracy: 0.9998 - val_loss: 9.8298e-04 - val_accuracy: 0.9997
Epoch 220/10000
13339/13339 [==============================] - 1

...

13339/13339 [==============================] - 20s 1ms/step - loss: 1.7118e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 0.9998
Epoch 2347/10000
13339/13339 [==============================] - 21s 2ms/step - loss: 1.8110e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9998
Epoch 2348/10000
13339/13339 [==============================] - 21s 2ms/step - loss: 1.6467e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9998
Epoch 2349/10000
13339/13339 [==============================] - 21s 2ms/step - loss: 1.8256e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 0.9998
Epoch 2350/10000
13339/13339 [==============================] - 20s 1ms/step - loss: 1.8560e-04 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 0.9998
Epoch 2351/10000
13339/13339 [==============================] - 22s 2ms/step - loss: 1.3734e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 0.9998
Epoch 2352/10000
13339/13339 [==============================] - 19s 1ms/ste

13339/13339 [==============================] - 19s 1ms/step - loss: 1.4010e-04 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 0.9998
Epoch 2399/10000
13339/13339 [==============================] - 20s 2ms/step - loss: 1.5016e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 2400/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 1.6889e-04 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 0.9998
Epoch 2401/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 2.0784e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 0.9998
Epoch 2402/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 2.2777e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9998
Epoch 2403/10000
13339/13339 [==============================] - 19s 1ms/step - loss: 1.6974e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 2404/10000
13339/13339 [==============================] - 19s 1ms/ste

13339/13339 [==============================] - 19s 1ms/step - loss: 2.1002e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 0.9998
Epoch 2451/10000
13339/13339 [==============================] - 20s 1ms/step - loss: 1.7727e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 0.9998
Epoch 2452/10000
13339/13339 [==============================] - 21s 2ms/step - loss: 1.6410e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 2453/10000
13339/13339 [==============================] - 20s 2ms/step - loss: 1.7730e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 2454/10000
13339/13339 [==============================] - 20s 2ms/step - loss: 1.4180e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 0.9998
Epoch 2455/10000
13339/13339 [==============================] - 21s 2ms/step - loss: 1.9119e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 2456/10000
13339/13339 [==============================] - 21s 2ms/ste

13339/13339 [==============================] - 19s 1ms/step - loss: 1.7889e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 2503/10000
 6050/13339 [============>.................] - ETA: 8s - loss: 1.5908e-04 - accuracy: 1.0000

KeyboardInterrupt: 

In [90]:
settings.model_folder_path

'/serialization/models/'

In [92]:
os.environ["CODE_PATH"]

'/Users/brian/code/'

In [95]:
model_path = os.path.join(os.environ["CODE_PATH"], 'oblak', 'next_oblak.h5')
# Serialize it
model.save(model_path)
del model
model = load_model(model_path)

# Validation (Micro)


In [12]:
# 100% accuracy
print(f'Accuracy: {model.evaluate(X_test, y_test, verbose = False)[1] * 100}%')
print(f'AUC Score: {roc_auc_score(y_test.flatten(), model.predict(X_test).flatten()) * 100}%')

Accuracy: 99.98325109481812%
AUC Score: 99.91822799877265%


In [40]:
rand_index = np.random.randint(len(X_test))
random_input = X_test[rand_index]

model_output = model.predict(random_input.reshape(1, n, n, 1)).reshape(n, n).round()

input_ptn = []
for row in random_input:
    if row.sum():
        input_ptn.append(int(row.sum()))

output_ptn = []
for row in model_output:
    if row.sum():
        output_ptn.append(int(row.sum())) 
    
original_ptn = Partition(input_ptn)
predicted_ptn = Partition(output_ptn)

print('ORIGINAL')
print(original_ptn)
print()

print('NEXT')
print(original_ptn._next_oblak_step[1])
print()

print('PREDICTED')
print(predicted_ptn)

ORIGINAL
▢▢▢▢▢▢▢
▢▢▢▢
▢▢▢▢
▢▢▢
▢▢▢
▢▢
▢

NEXT
▢▢▢▢▢
▢▢
▢

PREDICTED
▢▢▢▢▢
▢▢
▢


# Validation (Macro)

In [41]:
def macro_validate_partition(ptn_matrix, show = False):
    
    model_output = model.predict(ptn_matrix.reshape(1, n, n, 1)).reshape(n, n).round()

    input_ptn = []
    for row in ptn_matrix:
        if row.sum():
            input_ptn.append(int(row.sum()))

    output_ptn = []
    for row in model_output:
        if row.sum():
            output_ptn.append(int(row.sum())) 

    original_ptn = Partition(input_ptn)
    predicted_ptn = Partition(output_ptn)

    next_ptn = original_ptn._next_oblak_step[1]
    
    if show:
        
        print('ORIGINAL')
        print(original_ptn)
        print()

        print('NEXT')
        print(next_ptn._next_oblak_step[1])
        print()

        print('PREDICTED')
        print(predicted_ptn)
        
        
    return next_ptn == predicted_ptn

In [45]:
macro_validate_partition(Partition(28).fit_matrix(28), show = True)

ORIGINAL
▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢

NEXT
▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢

PREDICTED
▢
▢
▢


False

In [43]:
rights = []
wrongs = []
for matrix in X_test:
    if macro_validate_partition(matrix):
        rights.append(matrix)
        
    else:
        wrongs.append(matrix)
        
print(f'Macro Accuracy: {len(rights) / (len(wrongs) + len(rights)) * 100:.2f}%')

Macro Accuracy: 92.49%


In [44]:
len(rights) / (len(wrongs) + len(rights))

0.9248931864178097